In [10]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os
import keras as kb
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import roc_auc_score, accuracy_score

In [11]:
def load_images_and_labels(base_dir, target_size=(256, 256)):
    images = []
    labels = []  # 0 for 'Non_Cancer', 1 for 'Cancer'
    class_dirs = ['Non_Cancer', 'Cancer']
    
    for label, class_dir in enumerate(class_dirs):
        dir_path = os.path.join(base_dir, class_dir)
        for filename in os.listdir(dir_path):
            if filename.lower().endswith(('.jpg')):
                img_path = os.path.join(dir_path, filename)
                img = load_img(img_path, target_size=target_size, color_mode='rgb')
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(label)
                
    return np.array(images), np.array(labels)

# Adjust 'base_dir' to the path where your 'Skin_Data' folder is located
base_dir = 'Skin_Data'
images, labels = load_images_and_labels(base_dir)

# Normalize and reshape
images = images.astype('float32') / 255

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=123)


In [12]:
#early stopping
callbacks =[kb.callbacks.EarlyStopping(monitor = "val_accuracy", patience = 3, restore_best_weights=True)]

In [13]:
model = kb.Sequential([
    kb.layers.InputLayer(input_shape=(256, 256, 3)),
    kb.layers.Conv2D(32, (3, 3), activation='relu'),
    kb.layers.MaxPooling2D((2, 2)),
    kb.layers.Conv2D(64, (3, 3), activation='relu'),
    kb.layers.Conv2D(128, (3, 3), activation='relu'),
    kb.layers.MaxPooling2D((2, 2)),
    kb.layers.Conv2D(256, (3, 3), activation='relu'),
    kb.layers.Conv2D(128, (3, 3), activation='relu'),
    kb.layers.MaxPooling2D((2, 2)),
    kb.layers.Flatten(),
    kb.layers.Dense(100, activation='relu'),
    kb.layers.Dropout(0.2),
    kb.layers.Dense(10, activation='relu'),
    kb.layers.Dense(50, activation='relu'),
    kb.layers.Dropout(0.2),
    kb.layers.Dense(1, activation='sigmoid')
])

# compile  model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
# train model
model.fit(X_train, y_train, epochs=250,
          validation_data = (X_test, y_test),
          verbose=1)

model.summary()

Epoch 1/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 876ms/step - accuracy: 0.7002 - loss: 0.6371 - val_accuracy: 0.6897 - val_loss: 0.6200
Epoch 2/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 866ms/step - accuracy: 0.7291 - loss: 0.5965 - val_accuracy: 0.6897 - val_loss: 0.6257
Epoch 3/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 863ms/step - accuracy: 0.7154 - loss: 0.6194 - val_accuracy: 0.6897 - val_loss: 0.6275
Epoch 4/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 865ms/step - accuracy: 0.6975 - loss: 0.6205 - val_accuracy: 0.6897 - val_loss: 0.6161
Epoch 5/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 863ms/step - accuracy: 0.7152 - loss: 0.6247 - val_accuracy: 0.6897 - val_loss: 0.6146
Epoch 6/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 865ms/step - accuracy: 0.7141 - loss: 0.6078 - val_accuracy: 0.6897 - val_loss: 0.6256
Epoch 7/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 860ms/step - accuracy: 0.6901 - loss: 0.6274 - val_accuracy: 0.6897 - val_loss: 0.6211
Epoch 8/250
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 861ms/step - accuracy: 0.7143 - loss: 0.6035 - val_accuracy: 0.6897 - v

KeyboardInterrupt: 

In [ ]:
# Predict probabilities for the positive class
y_pred_probs = model.predict(X_train)

# Calculate AUC
auc_roc = roc_auc_score(y_train, y_pred_probs)

print(f"AUC-ROC training: {auc_roc}")

# Convert predicted probabilities to class labels
y_pred = (y_pred_probs > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_train, y_pred)

print(f"Accuracy: {accuracy}")

In [ ]:
# Predict probabilities for the positive class
y_pred_probs = model.predict(X_test)

# Calculate AUC
auc_roc = roc_auc_score(y_test, y_pred_probs)

print(f"AUC-ROC: {auc_roc}")

# Convert predicted probabilities to class labels
y_pred = (y_pred_probs > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

In [ ]:
keras.applications.ResNet101(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [ ]:
def load_images_and_labels(base_dir, target_size=(224, 224)):
    images = []
    labels = []  # 0 for 'Non_Cancer', 1 for 'Cancer'
    class_dirs = ['Non_Cancer', 'Cancer']

    for label, class_dir in enumerate(class_dirs):
        dir_path = os.path.join(base_dir, class_dir)
        for filename in os.listdir(dir_path):
            if filename.lower().endswith(('.jpg', '.png')):
                img_path = os.path.join(dir_path, filename)
                img = load_img(img_path, target_size=target_size, color_mode='rgb')
                img_array = img_to_array(img)
                img_array = preprocess_input(img_array)  # Preprocessing for ResNet50
                images.append(img_array)
                labels.append(label)

    return np.array(images), np.array(labels)

# Adjust 'base_dir' to the path where your 'Skin_Data' folder is located
base_dir = 'Skin_Data'
images, labels = load_images_and_labels(base_dir)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=123)

In [ ]:
# Define the model with ResNet50 base
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False  # Freeze the layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
]

In [ ]:

# Train the model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=callbacks, batch_size=32)

# Model summary
model.summary()

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

# Optional: Save the model
model.save('resnet50_skin_cancer_model.h5')